In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.expected_conditions import presence_of_element_located
from webdriver_manager.chrome import ChromeDriverManager 
import pandas as pd
from time import sleep

In [3]:
range_low = '35078000'
range_high = '35080000'
start_date = '01/08/2020'
end_date = '31/08/2020'

In [4]:
driver = webdriver.Chrome(ChromeDriverManager().install())

driver.get('https://www.hacienda.go.cr/tica/web/hcitrncm1.aspx')

[WDM] - Current google-chrome version is 85.0.4183
[WDM] - Get LATEST driver version for 85.0.4183


[WDM] - Driver [/home/andres/.wdm/drivers/chromedriver/linux64/85.0.4183.87/chromedriver] found in cache


In [5]:
img_src = driver.find_element_by_xpath('//div[@id="captchaImage"]/img')


In [6]:
img_src = img_src.get_attribute('src')
captchas = pd.read_csv('../Captchas/captchas.csv', index_col=0)
captcha_number = int(img_src.split('/')[-1].split('.')[0])
captcha_value = captchas.iloc[captcha_number - 1]
captcha_value

 captcha_answer    profit
Name: 13, dtype: object

In [7]:
driver.find_element_by_xpath('//select[@id="vVADUANA"]/option[@value="X"]').click()

In [8]:
date_i = driver.find_element_by_xpath('//div[@id="vVFCHIN_dp_container"]/input')

In [9]:
date_i.send_keys(start_date)

In [10]:
date_f = driver.find_element_by_xpath('//div[@id="vVFCHFN_dp_container"]/input')

In [11]:
date_f.send_keys(end_date)

In [12]:
range_start = driver.find_element_by_xpath('//td/input[@id="vVNCMINI"]')
range_start.clear()
range_start.send_keys(range_low)

In [13]:
range_end = driver.find_element_by_xpath('//td/input[@id="vVNCMFIN"]')
range_end.clear()
range_end.send_keys(range_high)

In [14]:
captcha_field = driver.find_element_by_xpath('//input[@id="_cfield"]')

In [15]:
captcha_field.clear()

In [16]:
captcha_field.send_keys(captcha_value)

In [17]:
driver.find_element_by_xpath('//input[@name="BUTTON1"]').click()


In [18]:
scrape = True
page = 1

while scrape:
    
    print(f'Page Number {page}')
    sleep(1)
    source = driver.page_source

    tables = pd.read_html(source)

    len_test = tables[9]
    
    sleep(1)

    scrape_pg = True
    
    while scrape_pg:
        sleep(1)
        source = driver.page_source

        tables = pd.read_html(source)

        main_pg_data = tables[9]
    
        duas_df = main_pg_data.groupby('Numero', sort=False).size()
        
        for i in range(1, len(duas_df) +1):
            sleep(2)
            line = duas_df.iloc[:i].sum()
            if (line != 10 and len(main_pg_data)//10 !=0) or (len(main_pg_data)//10 == 0):
                xpath = f'//table[@id="SubfileContainerTbl"]/tbody/tr/td[@colindex="2"]/span[@id="span_vVNUMECORRE_000{str(line)}"]/a'
                sleep(2)
                driver.find_element_by_xpath(xpath).click()
                sleep(2)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                if len(main_pg_data)//10 == 0:
                    scrape_pg = False
            elif line == 10:
                xpath = f'//table[@id="SubfileContainerTbl"]/tbody/tr/td[@colindex="2"]/span[@id="span_vVNUMECORRE_00{str(line)}"]/a'
                sleep(2)
                driver.find_element_by_xpath(xpath).click()
                sleep(2)
                ActionChains(driver).send_keys(Keys.ESCAPE).perform()
                sleep(1)
                scrape_pg = False
    
    if len(len_test)//10 != 0:
        sleep(1)
        page += 1
        driver.find_element_by_xpath('//input[@title="Siguiente"]').click()
        sleep(1)
    else:
        print('Job Done!')
        scrape = False

Page Numer 1
Page Numer 2
Page Numer 3
Page Numer 4
Page Numer 5
Page Numer 6
Job Done!


In [ ]:
main_pg_data

In [ ]:
duas_df

In [19]:
driver.quit()